In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib
import plotly.graph_objs as go
import plotly
import os
import seaborn as sns
from plotly.graph_objs import *
from ipywidgets import Image, HTML, Layout, Output, VBox, HBox
from IPython.display import Audio, display

In [2]:
IMAGES_FOLDER = 'spec_images/'

In [3]:
df_list=[]

for i in range(1,5):
    check = pd.read_json('data/nfile'+str(i)+'.json', orient ='split', compression = 'infer')
    df_list.append(check)

df = pd.concat(df_list)

In [4]:
image_data = {}
for img_filename in os.listdir(IMAGES_FOLDER):
    img_id = img_filename.split('.')[0]
    with open(IMAGES_FOLDER+img_filename, 'rb') as f:
        b = f.read()
        image_data[img_id] = b

In [5]:
raw_audio = df.raw_audio.values
samplerate = df.sr_hz.values
df = df.drop(columns=['raw_audio'])

In [6]:
x, y, z = df['UMAP1'], df['UMAP2'], df['UMAP3']
scat_labels = df['label']

labeltypes = sorted(list(set(scat_labels)))

# makes colors same as AVGN visualization
distinct_colors_14 = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231',
                      '#911eb4', '#46f0f0', '#f032e6', '#fffac8', 
                      '#008080', '#e6beff', '#9a6324', '#000000', '#aaffc3']
customPalette = sns.color_palette(distinct_colors_14, n_colors=len(labeltypes))
color_dict = dict(zip(labeltypes, customPalette))

c = [color_dict[val] for val in scat_labels]

In [7]:
trace = go.Scatter3d(x=x, y=y, z=z,
                     mode='markers',
                     hovertext = df['label'],
                     marker=dict(size=4,color=c,opacity=0.8))

In [9]:
data=[trace]

layout = go.Layout(scene=Scene(
                      xaxis=plotly.graph_objs.layout.scene.XAxis(title='UMAP1'),
                      yaxis=plotly.graph_objs.layout.scene.YAxis(title='UMAP2'),
                      zaxis=plotly.graph_objs.layout.scene.ZAxis(title='UMAP3')),
                  height = 700,
                  width = 700)

figure = go.Figure(data=data, layout=layout)

fig = go.FigureWidget(figure)

/Users/marathomas/opt/anaconda3/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:494: DeprecationWarning: plotly.graph_objs.Scene is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Scene

  DeprecationWarning,


In [10]:
scatter = fig.data[0]

scatter.text = df['index_id']
scatter.hoverinfo = 'text'

In [11]:
details = HTML()

In [12]:
image_widget = Image(
    value=image_data['20'],
    layout=Layout(height='252px', width='400px')
)

In [13]:
def play_audio(ind):
    display(Audio(raw_audio[ind], rate=samplerate[ind], autoplay=True))
    
audio_widget = Output()
audio_widget.layout.visibility = 'hidden'


def hover_fn(trace, points, state):

    ind = points.point_inds[0]
    
    # Update details HTML widget
    details.value = df.iloc[ind][['UMAP1', 'UMAP2', 'UMAP3', 'start', 'label', 'index_id']].to_frame().to_html()
    
    # Update image widget
    img_ind = df.iloc[ind]['index_id']
    image_widget.value = image_data[str(img_ind)]   

scatter.on_hover(hover_fn)

# play sound on click
def click_fn(trace, points, selector):

    ind = points.point_inds[0]
    with audio_widget:
        play_audio(ind)

scatter.on_click(click_fn)

In [14]:
HBox([
      VBox([details, image_widget, audio_widget]),
    fig])